<a href="https://colab.research.google.com/github/wwoskie/Biostat_BI_2022/blob/biostat_project_1/biostat_project_1/biostat_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>